In [1]:
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
from medmnist import INFO, Evaluator
from medmnist import PathMNIST

import numpy as np
import random
import torch
import timm
import os

/home/pmao/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def get_dataloaders(dataset_name, transform, batch_size=32, num_workers=4, download=True):    
    if dataset_name not in ['pathmnist']:
        raise ValueError("Dataset must be 'pathmnist'")
    
    dataset_class = PathMNIST
    
    train_dataset = PathMNIST(split="train", download=True, transform=transform)
    test_dataset = PathMNIST(split="test", download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    dataloaders = {
        'train' : train_loader,
        'test' : test_loader
    }
    
    return dataloaders


def extract_embeddings(model, device, dataloader):

    embeddings_db, labels_db = [], []

    for extracted in tqdm(dataloader):
        images, labels = extracted
        images = images.to(device)
        output = model.forward_features(images)
        output = model.forward_head(output, pre_logits=True)
        labels_db.extend(labels)
        embeddings_db.extend(output.detach().cpu().numpy())

    data = {
        'embeddings': embeddings_db,
        'labels': labels_db
    }

    return data

In [3]:
def create_database(dataset, backbone, save_dir, seed=42):

    seed_everything(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # get model from timm
    model = timm.create_model(backbone, pretrained=True, num_classes=0).to(device)
    model.requires_grad_(False) # remove?
    model = model.eval()
    
    # get the required transform function for the given feature extractor
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training=False)

    # get dataloaders and filenames
    dataloaders  = get_dataloaders(dataset, transforms)


    full_save_path = os.path.join(save_dir, dataset)
    # create database folders, if necessary
    os.makedirs(full_save_path, exist_ok=True)

    for split in ['train','test']:

        # get database of embeddings in the form
        #   db = {'embeddings' : [...], 'labels' : [...], 
        # the filenames are used for explainability purposes    
        db = extract_embeddings( model = model, 
                                 device = device,
                                 dataloader = dataloaders[split])
        
        # store database
        # database_root / dataset / train|test.npz
        np.savez(os.path.join(full_save_path,f'{split}.npz'), **db)

In [ ]:
backbone = "vit_base_patch14_dinov2.lvd142m"
dataset = "pathmnist"
save_dir = "/home/pmao/BachelorThesis/Deep_supervised_Hashing/PathMNIST/MedMNIST-PathMNIST"

create_database(dataset, backbone, save_dir)